# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from Kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications, we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

### Data Task Definition

Use regression techniques to predict the value of the target variable, price, based independent feature variables such as vehicle year, mileage, etc available in the dataset. This will be accomplished using by applying exploratory data analysis, feature engineering, and model selection to the provided dataset to identify important features which are predictors of sale price. The assumption is that this data analysis is for a 'standard' used car dealership that does not deal with exoctic or extremely vintage vehicles. 



### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn import set_config

# Set config to display the diagram
set_config(display='diagram')


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('data/vehicles.csv')
df.sample(5)

In [ ]:
df.describe(include='all').T.round(2)

In [ ]:
df.info()

In [ ]:
#assume id is unique for all columns, remove it before checking for duplicates
#Remove features that will not be usefull based on initial inspection: id, vin, model
df.drop(columns = ['id'], inplace=True)
duplicate_count = df.duplicated().sum()
print(f'Total number of duplicate rows: {duplicate_count}')
df = df.drop_duplicates()

In [ ]:
df.describe(include='all').round(2).T

In [ ]:
df['VIN'].value_counts()

In [ ]:
# Drop rows where 'VIN', 'price', and 'odometer' are the same
df = df.drop_duplicates(subset=['VIN', 'price', 'odometer'])

In [ ]:
df.describe(include='all').round(2).T

In [ ]:
#Remove features that will not be usefull based on initial inspection: vin, model
df.drop(columns = ['VIN', 'model'], inplace=True)

In [ ]:
df.describe(include='all').round(2).T

In [ ]:
#Looking at the year column, there appears to be some outliers since the min year is 1900. Even if these are not outliers, they are not useful for our analysis for a used car dealership. 
#Same goes for the price column, where the min price is 0 and very high max price.
#Remove rows where odometer is greater than 500000 miles or 0 miles (even new cars have greater than 0 miles)
df = df[df['year'] >= 1980]
df = df[(df['price'] >= 1000) & (df['price'] <= 150000)]
df = df[(df['odometer'] > 0) & (df['odometer'] <= 500000)]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (15, 4))
ax[0].hist(df['price'])
ax[0].grid()
ax[0].set_title('Original price')
ax[0].set_xlabel('Price')
ax[0].set_ylabel('Frequency')
ax[1].hist(np.log1p(df['price']))
ax[1].grid()
ax[1].set_title('Logarithm of price');
ax[1].set_ylabel('Frequency')
ax[1].set_xlabel('Log Price')

In [ ]:
# Create side-by-side plots of price and log price
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Histogram of original price
sns.histplot(df['price'], kde=True, ax=axes[0], bins=20, color='blue', edgecolor='black')
axes[0].set_title('Histogram and KDE of Price')
axes[0].set_xlabel('Price')
axes[0].set_ylabel('Density')

# Histogram of log-transformed price
sns.histplot(np.log(df['price']), kde=True, ax=axes[1], bins=20, color='green', edgecolor='black')
axes[1].set_title('Histogram and KDE of Log(Price)')
axes[1].set_xlabel('Log(Price)')
axes[1].set_ylabel('Density')

plt.tight_layout()
plt.show()

In [ ]:
#price appears to have a logarithmetic distribution add log(price) to dataframe
#df['log_price'] = np.log(df['price'])

In [ ]:
# Create scatter plots for numeric features
#numeric_features = ['year', 'odometer']
#categorical_features = ['condition', 'drive', 'transmission', 'fuel', 'type']
#for feature in numeric_features:
#        for categorical_feature in categorical_features:
#            plt.figure(figsize=(8, 6))
#            sns.scatterplot(x=feature, y='price', hue=categorical_feature, data=df)
#            plt.title(f'Scatter Plot of {feature} vs Price')
#            plt.xlabel(feature)
#            plt.ylabel('Price ($)')
#            plt.show()

### Data Preparation

After our initial exploration and fine-tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

### Take a closer look at remaining categorical features. Decide which ones have ordinality and will be converted to numeric features, which ones are suitible for one hot encoding and which ones should be dropped due to large dimensionality and/or missing features

In [ ]:
df['title_status'].unique()
df['title_status'].value_counts(dropna=False)

Since the vast majority of title_status values are 'clean' and the fact that most used car dealerships would not sell cars with other status values but would instead send them to an auction, the records with values other than clean will be removed and then the title_status column will be removed because it provides no value to linear regression model

In [ ]:
df = df[df['title_status'] == 'clean']
df.drop(columns = 'title_status', inplace=True)

In [ ]:
#find features with high percentage of missing values
(df.isna().sum() / len(df)) * 100

In [ ]:
df.isnull().sum().plot(kind='bar', figsize=(10, 6))
plt.title('Bar plot of missing values per feature')
plt.show()

In [ ]:
#Drop columns with more than 50% missing values - size is the only one
df.drop(columns = 'size', inplace=True)

In [ ]:
#examine the number of unqiue values for categorical features
for col in df.select_dtypes(include=['object', 'category']):
    print(f'{col}: # unique {len(df[col].unique())} % missing {round((df[col].isnull().sum()/len(df)) * 100)}')

In [ ]:
#Create barplot of average price by state because state has 50 unique values
plt.figure(figsize=(10, 6))
sns.barplot(x='state', y='price', data=df, estimator='mean', ci=None, palette='muted')
plt.xticks(rotation=45)
plt.title('Average Price by State')
plt.xlabel('State')
plt.ylabel('Average Price ($)')
plt.show()

In [ ]:
# Create box plots for categorical features - excluding state and region due to high dimensionality
categorical_features = ['drive', 'type', 'transmission', 'fuel', 'paint_color']

for feature in categorical_features:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=feature, y='price', data=df)
    plt.title(f'Box Plot of {feature} vs Price')
    plt.xlabel(feature)
    plt.ylabel('Price ($)')
    plt.show()

In [ ]:
# Initialize an empty DataFrame to store correlation values
correlation_table = pd.DataFrame()

#create a dataframes with desired categorical variables and examine their correlations with price
categorical_features = ['manufacturer', 'condition', 'cylinders', 'fuel', 'transmission', 'drive', 'paint_color', 'type', 'state']
for feature in categorical_features:
    print(f"Correlation of {feature} with price")
    categorical_df = pd.get_dummies(df[feature], dtype='int64')
    categorical_df['price'] = df['price']
    categorical_df.dropna(inplace=True)
    corr_matrix = categorical_df.corr()

    corr_with_price = corr_matrix['price'].drop('price').round(5)

    # Calculate the count of records for each category
    category_counts = categorical_df.drop(columns='price').sum()

    print(corr_with_price)
    print('\n') 

    temp_df = pd.DataFrame({
        'Feature': feature,
        'Category': corr_with_price.index,
        'Correlation with Price': corr_with_price.values,
        'Record Count': category_counts.values
    })
    # Concatenate the temporary DataFrame with the main correlation table
    correlation_table = pd.concat([correlation_table, temp_df], ignore_index=True)

    correlation_table.to_csv('categorical_correlation_report.csv', index=False)


In [ ]:
#The region and state columns are not useful for our analysis due to high dimensionality. We will drop these columns.
#There may be a correlation between the state and the price of the car, but we will not be able to use this information in a linear regression model.
#The 'type' is being dropped because it has relatively high dimensionality and relatively high percentatge of missing values.
df.drop(columns = ['region', 'type', 'state'], inplace=True)
df.describe(include='all').round(2).T

Since the number of cylinders is probably an important factor in the price of the vehicle, I want to keep the column even though 43% of the values are missing. In order to keep the feature it will be transformed to a numerical feature. For the missing cylinder values and the 'other' values, the average of the other values will be used.
The condition feature will also be converted to a numeric value

In [ ]:
#convert the number of cylinders to a numeric value
cylinder_map = {
    '6 cylinders': 6,
    '4 cylinders': 4,
    '8 cylinders': 8,
    '5 cylinders': 5,
    '10 cylinders': 10,
    '3 cylinders': 3,
    '12 cylinders': 12,
    'other': np.nan,   
    'NaN': np.nan 
}

df['cylinders_numeric'] = df['cylinders'].map(cylinder_map)
average_num_cylinders = df['cylinders_numeric'].mean()
df['cylinders_numeric'].fillna(average_num_cylinders, inplace=True)
df.drop(columns='cylinders', inplace=True)
df.rename(columns={'cylinders_numeric': 'cylinders'}, inplace=True)
df['cylinders'].value_counts()

In [ ]:
#convert the condition to a numeric value

#not interested in salvaged cars, drop them
df = df[df['condition'] != 'salvage']

condition_map = {
    'new': 5,
    'like new': 4,
    'excellent': 3,
    'good': 2,
    'fair': 1,  
    'NaN': np.nan  
}


df['condition'] = df['condition'].map(condition_map)
average_condition = df['condition'].mean()
df['condition'].fillna(average_condition, inplace=True)
df['condition'].value_counts()

In [ ]:
#remove categorical features based on analysis
#drop manufacturer because there are too many unique values and is not suitable for a linear regression model. It also has a low correlation
#drop paint_color and type because there are too many missing values and as well as too many unique values
#drop transmission, from scatter plot it is seen that the vast majority of values are automatic
df.drop(columns=['manufacturer', 'paint_color', 'transmission'], inplace=True)

In [ ]:
df['drive'].fillna('unknown', inplace=True)

In [ ]:
#for the type column, we will use the top 3 correlated values and assign all other values to the 'other' category
#top_types = ['pickup', 'truck', 'sedan']
#df['type'] = df['type'].apply(lambda x: x if x in top_types else 'other')

#Apply one-hot encoding
#df_encoded = pd.get_dummies(df, columns=['type'], dtype='int64')

#df_encoded.drop(columns='type_other', inplace=True)
#df = df_encoded


In [ ]:
df.sample(10)

In [ ]:
df.info()

In [ ]:
#drop remaining missing values
df.dropna(inplace=True)
df.describe().round(2).T

In [ ]:
df.info()

### Take a look at basic linear regression models for individual features to detect features with non-linear relationship to price

In [ ]:
#numeric_df = df.select_dtypes(include=['float64'])
#numeric_df['price'] = df['price']
#for col in numeric_df:
#    if(col != 'price'):
#        plt.figure(figsize=(10, 8))
#        sns.regplot(x=col, y='price', data=numeric_df)
#        plt.title(f'Price vs {col}')
#        plt.show()

In [ ]:
numeric_df = df.select_dtypes(include=['float64'])
numeric_df['price'] = df['price']
correlation_matrix = numeric_df.corr()
plt.figure(figsize=(12,12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', cbar=True, annot_kws={"size": 12})
plt.title('Correlation Matrix of Numeric Features')
plt.show()

In [ ]:
#correlation matrix for all numeric features
corr_matrix = numeric_df.corr()
corr_matrix['price'].sort_values(ascending=False)

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

In [ ]:
#X = df.drop(columns='price')
#y = np.log(df['price'])

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)  

In [ ]:
df.info()

In [ ]:
df.describe(include='all').T.round(2)

In [ ]:
#Convenience function provided by Jessica Cervi to calculate error metrics
def error_metrics(y_train_true, y_train_pred, y_test_true, y_test_pred):
    
    errors = {}
    
    # Errors for train data
    errors["Train_MAE"] = mean_absolute_error(y_train_true, y_train_pred)
    errors["Train_MSE"] = mean_squared_error(y_train_true, y_train_pred)
    errors["Train_RMSE"] = np.sqrt(errors["Train_MSE"])
    errors["Train_R2_Score"] = r2_score(y_train_true, y_train_pred)
    
    # Errors for test data
    errors["Test_MAE"] = mean_absolute_error(y_test_true, y_test_pred)
    errors["Test_MSE"] = mean_squared_error(y_test_true, y_test_pred)
    errors["Test_RMSE"] = np.sqrt(errors["Test_MSE"])
    errors["Test_R2_Score"] = r2_score(y_test_true, y_test_pred)
    
    return errors

In [ ]:
#Convenience function to assist performing simple cross-validation
def get_errors_for_degree_k_model(k, x_train, y_train, x_test, y_test):
    pipelined_model = Pipeline([
        ('poly_features', PolynomialFeatures(degree=k, include_bias=False)),
        ('scaler', StandardScaler()),
        ('linearRegression', LinearRegression(fit_intercept=True))
    ])
    pipelined_model.fit(x_train, y_train)
    y_train_pred = pipelined_model.predict(x_train)
    y_test_pred = pipelined_model.predict(x_test)

    return error_metrics(y_train, y_train_pred, y_test, y_test_pred)

In [ ]:
#simple cross-validation 
train_mses = []
test_mses = []
numeric_df = df[['year', 'condition', 'odometer', 'cylinders', 'price']]
#numeric_df = df[['year', 'odometer', 'price']]
X_numeric = numeric_df.drop(columns='price')
y_numeric = np.log(numeric_df['price'])
X_train_numeric, X_test_numeric, y_train_numeric, y_test_numeric = train_test_split(X_numeric, y_numeric, test_size=0.3, random_state=42) 
for i in range(1,11):
     errors = (get_errors_for_degree_k_model(i, X_train_numeric, y_train_numeric, X_test_numeric, y_test_numeric))
     train_mses.append(errors['Train_MSE'])
     test_mses.append(errors['Test_MSE'])
     print(errors)

In [ ]:
length = len(train_mses)
plt.title(f'The Complexity that minimized Test Error was: {test_mses.index(min(test_mses)) + 1}')
plt.suptitle('Simple Cross-Validation with scaling numeric features', fontsize=10, y=0.95)
plt.plot(range(1, (length +1)), train_mses, '--o', label = 'training error')
plt.plot(range(1, (length + 1)), test_mses, '--o', label = 'testing error')
plt.xticks(range(1, (length + 1)), range(1, (length + 1)))
plt.xlabel('Degree Complexity')
plt.ylabel('Mean Squared Error')
plt.legend()

In [ ]:
vehicles_df = pd.get_dummies(df, columns=['drive', 'fuel'], drop_first=True, dtype='int64')
vehicles_df.head()

In [ ]:
X = vehicles_df.drop(columns='price')
y = np.log(vehicles_df['price'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)  
selector = SequentialFeatureSelector(LinearRegression(), n_features_to_select=4)
best_features = selector.fit_transform(X_train, y_train)
best_features_df = pd.DataFrame(vehicles_df, columns = selector.get_feature_names_out())

best_features_df['price'] = vehicles_df['price']
best_features_df.sample(5)

In [ ]:
X = best_features_df.drop(columns='price')
y = np.log(best_features_df['price'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)  
for i in range(1,11):
     errors = (get_errors_for_degree_k_model(i, X_train, y_train, X_test, y_test))
     train_mses.append(errors['Train_MSE'])
     test_mses.append(errors['Test_MSE'])
     print(errors)

In [ ]:
pipe = Pipeline([('poly', PolynomialFeatures()), ('scale', StandardScaler()), ('ridge', Ridge())])
param_dict = {
    'ridge__alpha': [0.01, 0.1, 1, 10, 100],
    'poly__degree': [1, 2, 3, 4, 5, 6]
}
pipe

In [ ]:
X = best_features_df.drop(columns='price')
y = np.log(best_features_df['price'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) 

In [ ]:
grid = GridSearchCV(pipe, param_grid=param_dict)
grid.fit(X_train, y_train)
train_preds = grid.predict(X_train)
test_preds = grid.predict(X_test)

errors = error_metrics(y_train, train_preds, y_test, test_preds)
grid_params = grid.best_params_
#coefficients = grid_params.named_steps['ridge'].coef_



# Answer check
print(f'Train MSE: {errors["Train_MSE"]}')
print(f'Test MSE: {errors["Test_MSE"]}')
print(f'Train R2: {errors["Train_R2_Score"]}')
print(f'Test R2: {errors["Test_R2_Score"]}')
print(f'Best Degree: {list(grid_params.values())[0]}')
print(f'Best Alpha: {list(grid_params.values())[1]}')
#print(coefficients)

### Model Performance Analysis:
#### Simple Cross Validation Results
The table below shows the training and test Mean Squared Error and R2 Scores for the four cross validation models. The model that performed the best was model four which used the parameters determined using sequential feature selection. The test MSE was 0.240613496 and the R2 score was 0.646414436. However, this was only marginally better than the other models

#### GridSearchCV with Ridge Regression Results
The results from the hyperparameter tuning are shown in the table below. Overall the Ridge Regression performed slightly worse than Simple Cross Validation. With the best test MSE of 0.24615111472551 and R2 score of 0.638527812829079 for hyperparameters alpha = 0.01 and degree = 5. 



### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high-quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight into drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine-tuning their inventory.